# Public Key Cryptography

Learn asymmetric encryption using RSA public/private key pairs.

In [1]:
from pathlib import Path

from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA

# Search for key files starting from current directory and moving up
def find_project_root():
    current = Path.cwd()
    # Try current directory first
    if (current / "my_keypair").exists():
        return current
    # Try parent directory
    if (current.parent / "my_keypair").exists():
        return current.parent
    # Try two levels up (for nested structures)
    if (current.parent.parent / "my_keypair").exists():
        return current.parent.parent
    return current  # Default to current if not found

PROJECT_FOLDER = find_project_root()
print(f"Looking for keypairs in folder: {PROJECT_FOLDER}")

PRIVATE_KEY_FILE = PROJECT_FOLDER / "my_keypair"
PUBLIC_KEY_FILE = PROJECT_FOLDER / "my_keypair.pub"

assert PRIVATE_KEY_FILE.exists(), f"Private key not found at {PRIVATE_KEY_FILE}. CWD: {Path.cwd()}"
assert PUBLIC_KEY_FILE.exists(), f"Public key not found at {PUBLIC_KEY_FILE}"

print(f"✓ Found private key: {PRIVATE_KEY_FILE}")
print(f"✓ Found public key: {PUBLIC_KEY_FILE}")


Looking for keypairs in folder: /workspaces/ECBS-5147-Data-Engineering-Cloud-Computing-Managed-AI-Services
✓ Found private key: /workspaces/ECBS-5147-Data-Engineering-Cloud-Computing-Managed-AI-Services/my_keypair
✓ Found public key: /workspaces/ECBS-5147-Data-Engineering-Cloud-Computing-Managed-AI-Services/my_keypair.pub


Load the private key from file
The private key must be kept secret and secure
Extract the public key from our private key
The public key can be freely shared with anyone


In [2]:
with open(PRIVATE_KEY_FILE, "r", encoding="utf8") as key_file:
    private_key = RSA.import_key(key_file.read())

public_key = private_key.publickey()
print(f"Public key:\n{public_key.export_key().decode('utf-8')}")


Public key:
-----BEGIN PUBLIC KEY-----
MIIBojANBgkqhkiG9w0BAQEFAAOCAY8AMIIBigKCAYEAsvaUxMcpVmJNUzEG32yT
ci0UBzp5kkVFBHsUKI/tvZ6Kg0HETwYuA/okj3UEkQTpQRQV4EQ5zzHiWK4nVxi8
oDzP0ikb8NRjqBu2UZlvX21PEqYk5lS2KtRXsKauOBVZztRwac5vtSWTqhhdZ1Ui
zJ4Xo/PweX3h+FCl9WiRxYT2NLefSotGYqV/6CG4tHbzRa5z1Zo89CRT2rRv7CYr
j8SXde8nO4wIn7NO9qcO2o1zn1PEZ+FbS1hG4zlyerWao9VZW96If6TTVUhxpJ46
+f93k9P0x1b1A6ztX3NOdwYVCLdyAhxxOWvoPTMUuv5E4AAyUVnsnMjKXq8M3NE0
pTVnOZboHfDWtmCLsyjqHtcx6iOFfHE9rzgMdlh8ifbCq8CQflldF6BLNzqkqj53
Kqz7Q/LvRXmHgAFiH7OoGBEQ9vr/CxTI8HX2CBccFoLQj9Bpu5hf0/ell5EOfQc6
Xr1kjpFyMPBqCLDqt2MT/C5xhs58/e/N9SENH8KeJs2zAgMBAAE=
-----END PUBLIC KEY-----


Message to be encrypted - must be converted to bytes
Create a cipher object using the public key for encryption
Encrypt our message - only someone with the private key can decrypt it
Save the encrypted message to a file


In [3]:
short_secret_message = "My Secret Message".encode("utf-8")

public_key_cipher = PKCS1_OAEP.new(public_key)

encrypted_message = public_key_cipher.encrypt(short_secret_message)
print("Encrypted message:")
print(encrypted_message.hex())

ENCRYPTED_MESSAGE_FILE = PROJECT_FOLDER / "encrypted_message.bin"
with open(ENCRYPTED_MESSAGE_FILE, "wb") as f:
    f.write(encrypted_message)


Encrypted message:
4003e96acd8da236ec0ff79205394a1cda298b75fdfeca0a53cebee2d9f0d895b264b15647cdbcf62c0601b9c915b6bf20d035421b749f5f49ac093aa55b9c7ae33984f31217678c50bea52822d33da459da44557f2b2c7b4f839bf9486ac70e2d0ce9f329d4e5af09a785d92124a444175cacb89dd1754ad5465b751efd320d152ccd9bd5ca2d2affabd8a2b210730a7bd4e35487420750fc00151528d7b33117be6ed29148c747037169476d95164d864da9aa06ebe32d2446546ddfede3654a349f2365cc9110bf76f9c46f72c3fd29bb34c533f3611bf88239a9836593784f21f03eef373a1c6b45eb15e2829edb2daf29cd81a0e778a64f0042774d4ea5cec57e8560c58b53e5da80bb279a6156c310fbbb09d1fa7945c8e09b5b35016c9c858f3f10588d6a95455c8203eb9926fc6d33643b78d0e34aa2f4c89267178ab576cb0d45b3c90fba9cff51eeb7273f325668e77540f7d5099b5bb795449ced957c3cf757f890191c9822f415b03a30a4288f0ed9e7102b5579ec1a0b8e2cf9


Read the encrypted message from file
Create a cipher object using the private key for decryption
Decrypt the message using the private key
This demonstrates asymmetric encryption where:
1. Messages are encrypted with a public key and can only be decrypted with the matching private key
2. The public key can be freely shared
3. The private key must be kept secret
4. This is more secure than symmetric encryption as the decryption key (private key) is never shared


In [4]:
with open(ENCRYPTED_MESSAGE_FILE, "rb") as f:
    encrypted_message_from_file = f.read()

private_key_cipher = PKCS1_OAEP.new(private_key)

decrypted_message = private_key_cipher.decrypt(encrypted_message_from_file)
print(f"Decrypted message: {decrypted_message.decode('utf-8')}")



Decrypted message: My Secret Message
